# Deforestation Extraction (TMF & GFW Layers)

In [1]:
# load python packages
import os
import ee
import geemap
from glob import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Trigger the authentication flow
# need to do it when it is first time to run this notebook
# will be directed for permission confirmation 
# and The authorization workflow will generate a code, 
# which you should paste in the box below.
ee.Authenticate()

Enter verification code: 4/1ATx3LY60bvEmwOiSC3R8v_bpEgzxrnewJhakesXA_T4NlmG5Jk0iTG88dPo

Successfully saved authorization token.


In [3]:
# Initialize the library.
ee.Initialize()

### Create new empty Map to display

In [4]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Add shapefile to create reference area, project area and so on

### Input folder path and project area file name

In [5]:
## some parameters need to be provided
## replace them with your project info

# project start year
project_start_year = 2010

# baseline given in PD
baseline = 0

# actual observed in MR
observed = 0

# input project area file name, endwith .shp.
# replace folder name where you put project shapefile
project_area_path = glob(r'Project/AUD/902/*.shp')[0]

### pre-processing the project shapefile

In [ ]:
def crs_conversion(project_area_path):
    project_df = gpd.read_file(project_area_path)
    # print(f'original crs:{project_df.crs}')
    
    project_df_wgs84 = project_df.to_crs(epsg=4326)
    # print(f'new crs: {project_df_wgs84.crs}')
    
    centroid = project_df_wgs84.unary_union.centroid
    lon = centroid.x
    utm_zone = int((lon + 180) // 6) + 1
    epsg_number = 32600+utm_zone
    # print(f'new project crs: EPSG:326{utm_zone}')
    project_df_wgs84_p = project_df_wgs84.to_crs(epsg=epsg_number)
    # remove small polygons less than 1 ha
    project_df_wgs84_p['area'] = project_df_wgs84_p.area
    project_df_filter = project_df_wgs84_p[project_df_wgs84_p['area'] > 10000]
    
    return epsg_number, project_df_filter

In [ ]:
name, ext = os.path.splitext(os.path.basename(project_area_path))
new_pa_path = os.path.join(os.path.dirname(project_area_path), f'{name}_wgs84{ext}')
print(new_pa_path)

In [ ]:
epsg_number, project_df_filter = crs_conversion(project_area_path)
print(epsg_number)
project_df_filter.to_file(new_pa_path)

In [6]:
# upload the project shapefile to google earth engine
pa = geemap.shp_to_ee(new_pa_path)

# display project area in the map
Map.centerObject(pa, 10)
Map.addLayer(pa, {'color': 'blue'}, 'Project Area')

In [7]:
# create 5km or larger buffer around project area
# buffer distance can be changed as you want

distance = 5000
pa_buffer = pa.geometry().buffer(distance).difference(pa.geometry(), 1)

# display project area and buffer area together
Map.centerObject(pa, 10)
Map.addLayer(pa, {'color': 'blue'}, 'Project Area')
Map.addLayer(pa_buffer, {'color': 'red'}, 'Project Buffer Area')

In [8]:
# calculate project area and buffer area
pa_area = round(pa.geometry().area(1).getInfo()/10000, 2)
pa_buffer_area = round(pa_buffer.area(1).getInfo()/10000, 2)
print(f'Project area:{pa_area} ha, buffer area:{pa_buffer_area} ha')
factor = pa_area / pa_buffer_area
print(f'Scaling factor: {factor:.2f}')

Project area:1073847.84 ha, buffer area:701196.03 ha
Scaling factor: 1.53


### Tropical Moist Forest Dataset (TMF)

In [12]:
# tmf deforestation year and degradation year
# pixel value, no data, 0 (no deforestation) and deforestation year (1984-2024)
# make sure use the latest TMF dataset, change the dataset address if need,
# like, from v1_2024 to v1_2025.
tmf_deforestation_year = ee.ImageCollection('projects/JRC/TMF/v1_2024/DeforestationYear').mosaic()
tmf_degradation_year = ee.ImageCollection('projects/JRC/TMF/v1_2024/DegradationYear').mosaic()

In [13]:
# gt(a) or gte(a) force pixel value greater than (equal) a into 1,
# otherwise, return 0. ie., no loss and loss
tmf_deforestation = tmf_deforestation_year.gt(0.0)
tmf_degradation = tmf_degradation_year.gt(0.0)

In [15]:
# display: add tmf product deforestation and degradation year
Map = geemap.Map()

Map.addLayer(tmf_deforestation_year.updateMask(tmf_deforestation_year.gte(2010)),
            {'min': 2010,
             'max': 2024,
             'palette': ['yellow', 'red']},
             'TMF Deforestation Year 2010-2021')

Map.addLayer(tmf_degradation_year.updateMask(tmf_degradation_year.gte(2010)),
            {'min': 2010,
            'max': 2024,
            'palette': ['blue', 'green']},
            'TMF Degradation Year 2010-2021')

Map.centerObject(pa, 10)

Map.addLayer(pa,
            {'color': 'blue'},
            'Project Area')

Map.addLayer(pa_buffer,
            {'color': 'red'},
            'Project Buffer Area')
Map

Map(bottom=1079338.0, center=[-16.739800473128106, 29.235334686814006], controls=(WidgetControl(options=['posi…

In [10]:
# create a function to get annual deforestation or degradation in specific region (project area or buffer area)
# requires three parameters: loss image (value: 0, 1), loss year (value: 1986-2024) and region.
def get_annual_loss(loss_image, loss_year, region):
    loss_area = loss_image.multiply(ee.Image.pixelArea())
    loss_area_year = loss_area.addBands(loss_year).reduceRegion(
        **{'reducer': ee.Reducer.sum().group(**{'groupField': 1}),
           'geometry': region,
           'scale': 30,
           'bestEffort': True,
           'crs':f'EPSG:{epsg_number}',
           'maxPixels': 1e9})
    loss_area_year = loss_area_year.getInfo()['groups']
    years, annual_deforestation = [], []
    for g in loss_area_year:
        years.append(g['group'])
        annual_deforestation.append(g['sum']/10000)    
    return years, annual_deforestation

In [11]:
# get annual deforestation and degradation in pa
pa_years_def, pa_annual_deforestation = get_annual_loss(tmf_deforestation, tmf_deforestation_year.updateMask(tmf_deforestation_year), pa)
pa_years_deg, pa_annual_degradation = get_annual_loss(tmf_degradation, tmf_degradation_year.updateMask(tmf_degradation_year), pa)

In [12]:
# get annual deforestation and degradation in buffer
buffer_years_def, buffer_annual_deforestation = get_annual_loss(tmf_deforestation, 
                                                                tmf_deforestation_year.updateMask(tmf_deforestation_year), 
                                                                pa_buffer)
buffer_years_deg, buffer_annual_degradation = get_annual_loss(tmf_degradation, 
                                                              tmf_degradation_year.updateMask(tmf_degradation_year), 
                                                              pa_buffer)

In [13]:
df1 = pd.DataFrame({'Def_PA': pa_annual_deforestation}, index=pa_years_def)
df2 = pd.DataFrame({'Deg_PA': pa_annual_degradation}, index=pa_years_deg)
df3 = pd.DataFrame({'Def_Buffer': buffer_annual_deforestation}, index=buffer_years_def)
df4 = pd.DataFrame({'Deg_Buffer': buffer_annual_degradation}, index=buffer_years_deg)

In [14]:
data = pd.concat([df1, df2, df3, df4], ignore_index=False, axis=1, sort=True)
data

,Def_PA,Deg_PA,Def_Buffer,Deg_Buffer
1984,5.482762,17.407763,9.683775,134.163074
1985,4.089480,15.196414,4.199053,25.124860
1986,2.030371,9.018749,3.524498,22.423723
1987,8.313748,26.281566,19.302256,86.907020
1988,1.256025,45.428883,5.647209,112.011881
1989,0.566091,8.341873,3.049036,73.657544
1990,4.002159,11.744029,15.804458,60.822143
1991,0.039822,14.935575,2.695980,33.154724
1992,0.326112,8.772967,16.556197,56.807832
1993,0.655959,5.949923,71.973231,56.372074


In [15]:
data = data.fillna(0)
data.head()

,Def_PA,Deg_PA,Def_Buffer,Deg_Buffer
1984,5.482762,17.407763,9.683775,134.163074
1985,4.089480,15.196414,4.199053,25.124860
1986,2.030371,9.018749,3.524498,22.423723
1987,8.313748,26.281566,19.302256,86.907020
1988,1.256025,45.428883,5.647209,112.011881


In [16]:
# add more columns to dataframe
data['Def+Deg_PA'] = data['Def_PA'] + data['Deg_PA']
data['Def+Deg_Buffer'] = data['Def_Buffer'] + data['Deg_Buffer']

data['Def_Buffer_s'] = data['Def_Buffer'] * factor
data['Def_PA/Buffer_s'] = data['Def_PA']/data['Def_Buffer_s']
data['Def+Deg_Buffer_s'] = data['Def+Deg_Buffer'] * factor
data['Def+Deg_PA/Buffer_s'] = data['Def+Deg_PA']/data['Def+Deg_Buffer_s'] * factor

data.head()

,Def_PA,Deg_PA,Def_Buffer,Deg_Buffer,Def+Deg_PA,Def+Deg_Buffer,Def_Buffer_s,Def_PA/Buffer_s,Def+Deg_Buffer_s,Def+Deg_PA/Buffer_s
1984,5.482762,17.407763,9.683775,134.163074,22.890524,143.846849,5.399419,1.015436,80.205233,0.159131
1985,4.089480,15.196414,4.199053,25.124860,19.285894,29.323913,2.341282,1.746684,16.350245,0.657685
1986,2.030371,9.018749,3.524498,22.423723,11.049121,25.948221,1.965168,1.033180,14.468048,0.425814
1987,8.313748,26.281566,19.302256,86.907020,34.595314,106.209277,10.762432,0.772479,59.219509,0.325728
1988,1.256025,45.428883,5.647209,112.011881,46.684909,117.659091,3.148736,0.398898,65.603625,0.396781


In [17]:
data = data.round(2).fillna(0)
data.head()

,Def_PA,Deg_PA,Def_Buffer,Deg_Buffer,Def+Deg_PA,Def+Deg_Buffer,Def_Buffer_s,Def_PA/Buffer_s,Def+Deg_Buffer_s,Def+Deg_PA/Buffer_s
1984,5.48,17.41,9.68,134.16,22.89,143.85,5.40,1.02,80.21,0.16
1985,4.09,15.20,4.20,25.12,19.29,29.32,2.34,1.75,16.35,0.66
1986,2.03,9.02,3.52,22.42,11.05,25.95,1.97,1.03,14.47,0.43
1987,8.31,26.28,19.30,86.91,34.60,106.21,10.76,0.77,59.22,0.33
1988,1.26,45.43,5.65,112.01,46.68,117.66,3.15,0.40,65.60,0.40


In [21]:
data

,Def_PA,Deg_PA,Def_Buffer,Deg_Buffer,Def+Deg_PA,Def+Deg_Buffer,Def_Buffer_s,Def_PA/Buffer_s,Def+Deg_Buffer_s,Def+Deg_PA/Buffer_s
1994,0.0,0.00,0.00,0.98,0.00,0.98,0.00,0.0,0.87,0.00
1998,0.0,0.00,0.00,2.97,0.00,2.97,0.00,0.0,2.63,0.00
2000,0.0,0.09,0.00,1.88,0.09,1.88,0.00,0.0,1.66,0.05
2001,0.0,0.09,0.00,0.96,0.09,0.96,0.00,0.0,0.85,0.09
2002,0.0,0.00,0.00,0.63,0.00,0.63,0.00,0.0,0.55,0.00
2004,0.0,0.00,0.27,4.65,0.00,4.91,0.24,0.0,4.35,0.00
2005,0.0,0.00,1.52,56.85,0.00,58.37,1.34,0.0,51.65,0.00
2006,0.0,0.00,2.41,42.73,0.00,45.14,2.14,0.0,39.94,0.00
2007,0.0,2.97,0.09,5.23,2.97,5.32,0.08,0.0,4.71,0.56
2008,0.0,0.18,0.00,0.00,0.18,0.00,0.00,0.0,0.00,inf


In [19]:
# export to excel
# replace the file path and file name you want
data.to_excel('Project/AUD/2508/2508_TMF_Deforestation.xlsx')

### Global Forest Change dataset (GFC)

In [16]:
# load global forest change dataset image
# update to the latest version by changing the dataset address
gfc = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")

# select corresponding bands for analysis
# tree cover 2000
gfc_treecover_2000 = gfc.select(['treecover2000'])

# tree loss 
gfc_loss_image = gfc.select(['loss'])

# tree loss year (range from 1 to 21), base year is 2000.
gfc_loss_year = gfc.select(['lossyear'])

In [17]:
# Display GFC dataset and project area
Map = geemap.Map()
Map.addLayer(gfc_treecover_2000.updateMask(gfc_treecover_2000),
            {'palette': ['000000', '00FF00'], 'max': 100},
            'GFC Tree Cover 2000')
Map.addLayer(gfc_loss_image.updateMask(gfc_loss_image),
            {'palette': ['FF0000']},
            'GFC Tree Loss')

Map.addLayer(gfc_loss_year.updateMask(gfc_loss_year.gte(10)),
            {'min': 10,
            'max': 23,
            'palette': ['yellow', 'red']},
            'Tree loss after 2010')

Map.centerObject(pa,8)

Map.addLayer(pa,
            {'color': 'red'},
            'Project Area')

Map.addLayer(pa_buffer,
            {'color': 'blue'},
             'Project Area buffer')
Map

In [21]:
# extract deforestion in pa and buffer using GFC
pa_years_def_gfc, pa_annual_deforestation_gfc = get_annual_loss(gfc_loss_image, 
                                                                gfc_loss_year, 
                                                                pa)

pa_buffer_years_def_gfc, pa_buffer_annual_deforestation_gfc = get_annual_loss(gfc_loss_image, 
                                                                              gfc_loss_year, 
                                                                              pa_buffer)

In [22]:
df_gfc_1 = pd.DataFrame({'Def_PA': pa_annual_deforestation_gfc}, index=[d+2000 for d in pa_years_def_gfc])
df_gfc_2 = pd.DataFrame({'Def_Buffer': pa_buffer_annual_deforestation_gfc}, index=[d+2000 for d in pa_buffer_years_def_gfc])

In [23]:
data_gfc = pd.concat([df_gfc_1, df_gfc_2], ignore_index=False, axis=1, sort=True)
data_gfc.head()

,Def_PA,Def_Buffer
2001,17.165839,1116.319134
2002,55.175627,787.667147
2003,16.866597,1626.751405
2004,6.931363,1214.566148
2005,19.133270,3410.280712


In [24]:
data_gfc = data_gfc.round(2).fillna(0)
data_gfc.head()

,Def_PA,Def_Buffer,Def_Buffer_s,Def_PA/Buffer_s
2001,17.17,1116.32,622.43,0.03
2002,55.18,787.67,439.18,0.13
2003,16.87,1626.75,907.03,0.02
2004,6.93,1214.57,677.21,0.01
2005,19.13,3410.28,1901.48,0.01


In [25]:
# export to excel file
# replace file path and file name as you want
data_gfc.to_excel('Project/AUD/2508/2508_GFC_Deforestation.xlsx')